### Time Series Workshop 
# 7. Multi-Step Ahead Forecasting &#x1F449; &#x1F449; &#x1F449;

For now, we've limited ourselves to single-step forecasting, i.e., we always predicted one given time-step (1h for the air pollution, 1month for the retail challenge) ahead.

But what about multi-step forecasting. Can we predict the next 24h of air pollution? Or the next 12 months of retail sales?

Here we'll tackle this problem and dive into the two most common approaches to multi-step forecasting: 
- Direct forecasting
- Recursive forecasting 

In [ ]:
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

from timeseries.data import load_airline
from timeseries.utils import print_metrics

DATA_DIR = Path("..") / Path("data")

## Load data
Let's look at a simple example: Monthly airline passengers in the US!

In [ ]:
SPLIT_DATE = "1958-01-01"
TARGET_COL = "passengers"

df_in = load_airline(DATA_DIR / Path("airline_passengers.csv"))
df_in.head()

In [ ]:
_ = df_in.plot(figsize=(12, 3))

df = df_in.copy()

df_train = df[df.index <= SPLIT_DATE]
df_test = df[df.index > SPLIT_DATE]

## Multi-step forecasting: Direct approach!
Let's first tackle the so-called "direct" approach to multi-step forecasting

In the direct approach, we build a model to predict each one of the steps in the forecasting horizon.

This means that we need to create suitable targets first:

- 1 month ahead
- 2 months ahead
- 3 months ahead
- ...
- 12 months ahead

Sounds expensive? That's because it is!


<img src="../images/skforecast_direct.png" width="800">

Source: [skforecast.org](https://skforecast.org/0.7.0/index.html)

The little-known package skforecast has some easy to use wrappers for that!

In [ ]:
from skforecast.direct import ForecasterDirect

forecaster = ForecasterDirect(
    regressor=Lasso(alpha=3, random_state=0), steps=len(df_test), lags=15
)

forecaster.fit(y=df_train[TARGET_COL])

df_pred = pd.DataFrame(
    forecaster.predict(steps=len(df_test)).values,
    index=df_test.index,
    columns=[TARGET_COL],
)

_, ax = plt.subplots(figsize=(12, 3))
_ = df_train.plot(ax=ax)
_ = df_test.plot(ax=ax)
_ = df_pred.plot(ax=ax)
_ = ax.legend(["train", "test", "pred"])

print_metrics(df_test, df_pred)

## Multi-step forecasting: Recursive approach!

In contrast to the direct approach, we only need a single trained model for the recursive forecast.

Here, we iterate over single forecasting steps and use these forecasts for the next single-step of forecasts until we reach the end of our horizon.

One major disadvantage compared to the direct approach could be, that forecasting errors quickly add up.


<img src="../images/skforecast_recursive.png" width="800">

Source: [skforecast.org](https://skforecast.org/0.7.0/index.html)

In [ ]:
from skforecast.recursive import ForecasterRecursive

forecaster = ForecasterRecursive(regressor=Lasso(alpha=3, random_state=0), lags=15)
forecaster.fit(y=df_train[TARGET_COL])

df_pred = pd.DataFrame(
    forecaster.predict(steps=len(df_test)).values,
    index=df_test.index,
    columns=[TARGET_COL],
)

_, ax = plt.subplots(figsize=(12, 3))
_ = df_train.plot(ax=ax)
_ = df_test.plot(ax=ax)
_ = df_pred.plot(ax=ax)
_ = ax.legend(["train", "test", "pred"])

print_metrics(df_test, df_pred)

Eh, the jury is out on what approach works better.
Here's the recursive stratgegy explained a bit more thoroughly:

<img src="../images/udemy_recursive_1.png" width="500"><img src="../images/udemy_recursive_2.png" width="500">

<img src="../images/udemy_recursive_3.png" width="500"><img src="../images/udemy_recursive_4.png" width="500">

Source: [Udemy: Feature Engineering for Time Series Forecasting](https://www.udemy.com/course/feature-engineering-for-time-series-forecasting/).


## Multi-step forecasting: Direct instructive approach!

These models are nice to use, but maybe we can built a bit more instructive example by hand.

The number of time steps we want to forecast is also called the forecasting **"horizon"**.

In [ ]:
# Create an empty dataframe for the new targets.
df_train_multi = pd.DataFrame(index=df_train.index)
df_test_multi = pd.DataFrame(index=df_test.index)

# Add each one of the steps ahead.
for h in range(len(df_test)):
    df_train_multi[f"m_{h}"] = df_train.shift(periods=-h, freq="MS")
    df_test_multi[f"m_{h}"] = df_test.shift(periods=-h, freq="MS")

df_train_multi.head(3)

Now, with the **MultiOutputRegressor** wrapper from Scikit-Learn we can automatically create one regression per target:

In [ ]:
from feature_engine.datetime import DatetimeFeatures
from sklearn.multioutput import MultiOutputRegressor

df_train_multi.dropna(inplace=True)
df_test_multi.dropna(inplace=True)

# Date feature transformer:
datetime_features = DatetimeFeatures(
    variables="month",
    features_to_extract=["month", "year"],
)

pipe = Pipeline(
    [("dt", datetime_features), ("regressor", Lasso(alpha=2, random_state=0))]
)

model = MultiOutputRegressor(pipe)
model.fit(X=df_train_multi.reset_index()[["month"]], y=df_train_multi)

In [ ]:
preds = model.predict(df_test_multi.reset_index()[["month"]])
df_pred = pd.DataFrame(preds[0], columns=df_test.columns, index=df_test.index)

_, ax = plt.subplots(figsize=(12, 3))
_ = df_train.plot(ax=ax)
_ = df_test.plot(ax=ax)
_ = df_pred.plot(ax=ax)
_ = ax.legend(["train", "test", "pred"])

print_metrics(df_test, df_pred)